# Importing

In [119]:
import requests
import json
import os
import modules.config as config
from time import sleep
from requests.auth import HTTPBasicAuth
from ibm_watson import DiscoveryV1
from pprint import pprint
import modules.discovery_helper as dh_help
import modules.common_helper as cm_help
import sys
import pandas as pd
import modules.config as config
import math
import re
import hashlib
import unicodedata
from bs4 import BeautifulSoup 
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

# Creating instance


In [120]:
all_params = cm_help.get_params()
COLLECTION_NAME = config.DISCOVERY_COLLECTION_NAME
print(COLLECTION_NAME)

Test


In [121]:
all_params['discovery']

{'default': 'a1',
 'r3': {'version': '2019-04-30',
  'apikey': 'e60nrAsq6WIQmWfuzv9_8by2yCXinHK_FOe3u9MSI1ml',
  'url': 'https://api.us-east.discovery.watson.cloud.ibm.com/instances/f2df1dc1-c923-49c1-80f3-7e23fc0441e8'},
 'a1': {'version': '2019-04-30',
  'apikey': 'h1IoNg4AIKbGtBs8n8QGf4dZ5tZwMpeVERQUrC1Q2-DF',
  'url': 'https://api.jp-tok.discovery.watson.cloud.ibm.com/instances/2330122b-7269-496b-ba7e-029f641470a1'},
 'version1': '2018-12-03',
 'apikey1': 'LTk2AGh0Mjsu3G9bGGSkU92xeCBAYdfa4lN4fjaA-CR4',
 'url1': 'https://gateway.watsonplatform.net/discovery/api'}

In [122]:
def discovery_init(COLLECTION_NAME=config.DISCOVERY_COLLECTION_NAME,default=all_params['discovery']['default']):
    response = {}
    
    discovery_param = all_params['discovery']
    # default = 
    authenticator = IAMAuthenticator(apikey=discovery_param[default]['apikey'])
    endurl = "https://gateway.watsonplatform.net/discovery/api"
    discovery = DiscoveryV1(
        version=discovery_param[default]['version'],
        authenticator = authenticator)
    iam_apikey=discovery_param[default]['apikey'],
    url=discovery_param[default]['url']    
    if default =="r3":
        discovery.set_service_url(discovery_param[default]['url'])

    environments = discovery.list_environments().get_result()
    # print(json.dumps(environments, indent=2))
    print("Discovery instance loaded "+default)
    js = json.dumps(environments)
    jres = json.loads(js)
    # print(jres['environments'][1]['environment_id'])
    env_id = jres['environments'][1]['environment_id']
    collections = discovery.list_collections(env_id).get_result()
    cols = json.dumps(collections, indent=2)
    colres = json.loads(cols)

    #print(colres['collections'])
    for item in colres['collections']:
        if item['name']== COLLECTION_NAME:
            print('COLLECTION ID:', item['collection_id'], 'COLLECTION NAME:', item['name'])
            col_id = item['collection_id']
    response['env_id'] = env_id
    response['col_id'] = col_id
    response['discovery'] = discovery
    return response

In [123]:
discovery_config = discovery_init(COLLECTION_NAME='Naviga1',default="r3")
#COLLECTION_NAME = 'Naviga1'
#discovery_config = dh_help.discovery_init(COLLECTION_NAME=COLLECTION_NAME,default="r3")

Discovery instance loaded r3
COLLECTION ID: ec586f27-f848-40b7-99af-dab80a43b248 COLLECTION NAME: Naviga1


# NULL query

In [11]:
offset = 0
count = 10
reqparams = {'offset':offset,'count':count}
discovery_data = dh_help.getDiscoveryData(discovery_config,reqparams)

offset in query = 0


# Getting html text

In [125]:
offset = 0
count = 20
reqparams = {'offset':offset,'count':count}
discovery_data = dh_help.getDiscoveryData(discovery_config,reqparams)
print("len "+str(len(discovery_data['results'])))
print(discovery_data['results'][10]['id'])
print(discovery_data['results'][10]['html'])

offset in query = 0
len 20
63f3d564-14f3-4909-af49-10efdeb0cb20

<p><location idsrc="xmltag.org" value="LU/us.ny.buffal">BUFFALO, N.Y.</location> (WIVB) - <chron>Tuesday, June 2</chron> , <chron>7:56 p.m.</chron></p>
<p>A <location idsrc="xmltag.org" value="LU/us.ny.buffal">Buffalo</location> woman has been charged in connection with the incident that injured three law enforcement officers Monday night.</p>

<p>State police arrested 30-year-old <person>Deyanna Davis</person> after she was released from <org idsrc="xmltag.org" value="ACORN:2148143717">Erie County Medical Center</org>.</p>

<p>Davis was driving the car that went through a line of officers at a protest on <location>Bailey Avenue</location> hitting three officers, seriously injuring one.</p>

<p>Davis is facing multiple felony charges including aggravated assault on a police officer and criminal possession of a weapon.</p>

<p><location idsrc="xmltag.org" value="LU/us.ny.buffal">Buffalo</location> woman charged in connecti

# REGEX

In [166]:
source =discovery_data['results'][10]['html']

In [100]:
batRegex = re.compile(r'([a-zA-Z]{2}\s<[a-zA-Z]*[^>]*>(.+?)</[a-zA-Z]*>)')
m1=batRegex.search(source)

In [101]:
print (re.findall(batRegex,source))

[('on <chron>Wednesday, June 3</chron>', 'Wednesday, June 3'), ('to <org>Sweetwater County Public Health</org>', 'Sweetwater County Public Health'), ('th <org>Sweetwater County Public Health</org>', 'Sweetwater County Public Health'), ('to <org>Sweetwater County Public Health</org>', 'Sweetwater County Public Health'), ("ct <org>Memorial Hospital of Sweetwater County's</org>", "Memorial Hospital of Sweetwater County's"), ('nd <org>Castle Rock Medical Center</org>', 'Castle Rock Medical Center'), ('at <a href="http://sweetwater311.org">sweetwater311.org</a>', 'sweetwater311.org'), ('nd <a href="http://cdc.gov">cdc.gov</a>', 'cdc.gov')]


In [102]:
lists = re.findall(batRegex,source)

In [103]:
len_ = len(lists)

In [104]:
for i in range(len_):
    lists[i] = list(lists[i])

In [105]:
lists

[['on <chron>Wednesday, June 3</chron>', 'Wednesday, June 3'],
 ['to <org>Sweetwater County Public Health</org>',
  'Sweetwater County Public Health'],
 ['th <org>Sweetwater County Public Health</org>',
  'Sweetwater County Public Health'],
 ['to <org>Sweetwater County Public Health</org>',
  'Sweetwater County Public Health'],
 ["ct <org>Memorial Hospital of Sweetwater County's</org>",
  "Memorial Hospital of Sweetwater County's"],
 ['nd <org>Castle Rock Medical Center</org>', 'Castle Rock Medical Center'],
 ['at <a href="http://sweetwater311.org">sweetwater311.org</a>',
  'sweetwater311.org'],
 ['nd <a href="http://cdc.gov">cdc.gov</a>', 'cdc.gov']]

In [167]:
batRegex = re.compile(r'(.<[a-zA-Z]*[^>]*>(.+?)</[a-zA-Z]*>)')
m1=batRegex.search(source)

In [168]:
print (re.findall(batRegex,source))

[('><location idsrc="xmltag.org" value="LU/us.ny.buffal">BUFFALO, N.Y.</location>', 'BUFFALO, N.Y.'), (' <chron>Tuesday, June 2</chron>', 'Tuesday, June 2'), (' <chron>7:56 p.m.</chron>', '7:56 p.m.'), (' <location idsrc="xmltag.org" value="LU/us.ny.buffal">Buffalo</location>', 'Buffalo'), (' <person>Deyanna Davis</person>', 'Deyanna Davis'), (' <org idsrc="xmltag.org" value="ACORN:2148143717">Erie County Medical Center</org>', 'Erie County Medical Center'), (' <location>Bailey Avenue</location>', 'Bailey Avenue'), ('><location idsrc="xmltag.org" value="LU/us.ny.buffal">Buffalo</location>', 'Buffalo'), (' <a href="http://pic.twitter.com/vBhRDNynGl">pic.twitter.com/vBhRDNynGl</a>', 'pic.twitter.com/vBhRDNynGl'), (' <chron>June 2, 2020</chron>', 'June 2, 2020'), ('><chron>Tuesday, June 2</chron>', 'Tuesday, June 2'), (' <chron>1:13 p.m.</chron>', '1:13 p.m.'), (' <org>New York State Police</org>', 'New York State Police'), (' <location>Bailey Ave.</location>', 'Bailey Ave.'), (' <person>

In [169]:
lists = re.findall(batRegex,source)

In [170]:
len_ = len(lists)
for i in range(len_):
    lists[i] = list(lists[i])

In [172]:
lists

[['><location idsrc="xmltag.org" value="LU/us.ny.buffal">BUFFALO, N.Y.</location>',
  'BUFFALO, N.Y.'],
 [' <chron>Tuesday, June 2</chron>', 'Tuesday, June 2'],
 [' <chron>7:56 p.m.</chron>', '7:56 p.m.'],
 [' <location idsrc="xmltag.org" value="LU/us.ny.buffal">Buffalo</location>',
  'Buffalo'],
 [' <person>Deyanna Davis</person>', 'Deyanna Davis'],
 [' <org idsrc="xmltag.org" value="ACORN:2148143717">Erie County Medical Center</org>',
  'Erie County Medical Center'],
 [' <location>Bailey Avenue</location>', 'Bailey Avenue'],
 ['><location idsrc="xmltag.org" value="LU/us.ny.buffal">Buffalo</location>',
  'Buffalo'],
 [' <a href="http://pic.twitter.com/vBhRDNynGl">pic.twitter.com/vBhRDNynGl</a>',
  'pic.twitter.com/vBhRDNynGl'],
 [' <chron>June 2, 2020</chron>', 'June 2, 2020'],
 ['><chron>Tuesday, June 2</chron>', 'Tuesday, June 2'],
 [' <chron>1:13 p.m.</chron>', '1:13 p.m.'],
 [' <org>New York State Police</org>', 'New York State Police'],
 [' <location>Bailey Ave.</location>', 'Bai

In [173]:
regex = re.compile(r'\svalue="(.*?)"')
subst = ''
for i in range(len_):
    bregex = re.findall(regex,lists[i][0])
    if(len(bregex)!=0):
        lists[i][0] = lists[i][0].replace(bregex[0], subst)

regex = re.compile(r'\sidsrc="(.*?)"')
for i in range(len_):
    bregex = re.findall(regex,lists[i][0])
    if(len(bregex)!=0):
        lists[i][0] = lists[i][0].replace(bregex[0], subst)

regex = re.compile(r'\shref="(.*?)"')
for i in range(len_):
    bregex = re.findall(regex,lists[i][0])
    if(len(bregex)!=0):
        lists[i][0] = lists[i][0].replace(bregex[0], subst)
    

In [174]:
lists

[['><location idsrc="" value="">BUFFALO, N.Y.</location>', 'BUFFALO, N.Y.'],
 [' <chron>Tuesday, June 2</chron>', 'Tuesday, June 2'],
 [' <chron>7:56 p.m.</chron>', '7:56 p.m.'],
 [' <location idsrc="" value="">Buffalo</location>', 'Buffalo'],
 [' <person>Deyanna Davis</person>', 'Deyanna Davis'],
 [' <org idsrc="" value="">Erie County Medical Center</org>',
  'Erie County Medical Center'],
 [' <location>Bailey Avenue</location>', 'Bailey Avenue'],
 ['><location idsrc="" value="">Buffalo</location>', 'Buffalo'],
 [' <a href="">pic.twitter.com/vBhRDNynGl</a>', 'pic.twitter.com/vBhRDNynGl'],
 [' <chron>June 2, 2020</chron>', 'June 2, 2020'],
 ['><chron>Tuesday, June 2</chron>', 'Tuesday, June 2'],
 [' <chron>1:13 p.m.</chron>', '1:13 p.m.'],
 [' <org>New York State Police</org>', 'New York State Police'],
 [' <location>Bailey Ave.</location>', 'Bailey Ave.'],
 [' <person>Ronald Ensminger Jr</person>', 'Ronald Ensminger Jr'],
 [' <person>Randall Shenefiel</person>', 'Randall Shenefiel'],


In [165]:
lists[8][0].rsplit('/',1)

[' <a href="">pic.twitter.com/vBhRDNynGl<', 'a>']

In [175]:
res = {}
for i in range(len_):
    lists[i][0]=lists[i][0].rsplit('/',1)
    lists[i][0][1]=lists[i][0][1].rsplit('>',1)
    keyword = lists[i][0][1][0]
    value = lists[i][1]
    
    if keyword in res:
        if value not in res[keyword]:
            res[keyword].append(value)       
    else:
        res[keyword] = [value]

In [176]:
res

{'location': ['BUFFALO, N.Y.',
  'Buffalo',
  'Bailey Avenue',
  'Bailey Ave.',
  'Erie County',
  'Baliey Avenue',
  'West Seneca',
  'New York State',
  'Route 33'],
 'chron': ['Tuesday, June 2',
  '7:56 p.m.',
  'June 2, 2020',
  '1:13 p.m.',
  '12:54 p.m.',
  'June 22',
  '9:30 a.m.',
  '10 p.m. Monday night'],
 'person': ['Deyanna Davis',
  'Ronald Ensminger Jr',
  'Randall Shenefiel',
  'Semaj Pigram',
  'Walter Stewart Jr',
  'John Flynn',
  'Decker Street Monday',
  'Dave Greber'],
 'org': ['Erie County Medical Center',
  'New York State Police',
  'State Police'],
 'a': ['pic.twitter.com/vBhRDNynGl', 'pic.twitter.com/YQWiJim85f'],
 'money': ['$100,000', '$175,000']}

# Error handling

In [ ]:
discovery_data = dh_help.discovery_init(COLLECTION_NAME='Naviga1')

In [ ]:
import json
import os
import sys
sys.path.append("../")
sys.path.append("../../")
sys.path.append("../modules/")
import modules.common_helper as cm_help
from ibm_watson import DiscoveryV1
import modules.config as config
import modules.discovery_helper as dh
import os
import time
from pprint import pprint
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

In [ ]:
all_params = cm_help.get_params()
COLLECTION_NAME = config.DISCOVERY_COLLECTION_NAME
print(COLLECTION_NAME)

In [ ]:
all_params['discovery']

In [ ]:
def discovery_init(COLLECTION_NAME=config.DISCOVERY_COLLECTION_NAME,default=all_params['discovery']['default']):
    response = {}
    
    discovery_param = all_params['discovery']
    # default = 
    authenticator = IAMAuthenticator(apikey=discovery_param[default]['apikey'])
    endurl = "https://gateway.watsonplatform.net/discovery/api"
    discovery = DiscoveryV1(
        version=discovery_param[default]['version'],
        authenticator = authenticator)
    iam_apikey=discovery_param[default]['apikey'],
    url=discovery_param[default]['url']    
    if default =="r3":
        discovery.set_service_url(discovery_param[default]['url'])

    environments = discovery.list_environments().get_result()
    # print(json.dumps(environments, indent=2))
    print("Discovery instance loaded "+default)
    js = json.dumps(environments)
    jres = json.loads(js)
    # print(jres['environments'][1]['environment_id'])
    env_id = jres['environments'][1]['environment_id']
    collections = discovery.list_collections(env_id).get_result()
    cols = json.dumps(collections, indent=2)
    colres = json.loads(cols)

    #print(colres['collections'])
    for item in colres['collections']:
        if item['name']== COLLECTION_NAME:
            print('COLLECTION ID:', item['collection_id'], 'COLLECTION NAME:', item['name'])
            col_id = item['collection_id']
    response['env_id'] = env_id
    response['col_id'] = col_id
    response['discovery'] = discovery
    return response

In [ ]:
discovery_data = discovery_init(COLLECTION_NAME='Naviga1',default="r3")

In [ ]:
dict_ = []
result = discovery_data['results']

for obj in result:
    temp={}
    temp['id'] = obj['id']

    # dict for 0th doc
    res = {}
    for tag in soup.findAll('p'):
        #tag.find(x for x in ent_types)
        for ent in ent_types:
            try:
                tag.find(ent)
                res[ent].append(tag.find(ent).text)
            except Exception as e:
                pass
    # for extra tags not in ent_types
    for tag in soup.findAll('chron'):
        res['chron'].append(tag.text)
    
    temp = {**temp,**res}
    dict_.append(temp)